In [1]:
import psycopg2 # need install
import json

In [2]:
# loading from config file
with open("config.json", "r") as file:
    config = json.load(file)


In [3]:
db_config = config["db"]
conn = psycopg2.connect(
            host=db_config["host"],
            dbname=db_config["dbname"],
            user=db_config["user"],
            password=db_config["pwd"]
    )
if conn:
    print("Connected to database %s sucessfully!" % db_config["dbname"])
else:
    print("Connection failed.")

Connected to database db4031 sucessfully!


```python
#query with template
query_temp = "SELECT * FROM nation WHERE nation.n_regionkey = %s;"
vals = (1,)
query = cursor.mogrify(query_temp, vals)
```

```python
# Explain params?
cursor.execute("EXPLAIN (analyze, buffers, verbose, format json) " + query)
cursor.execute("EXPLAIN (FORMAT JSON) " + query)
```

In [4]:
if conn:
    cursor = conn.cursor()
    query = "SELECT * FROM customer as c, nation as n  WHERE c.c_nationkey = n.n_nationkey"
    cursor.execute("EXPLAIN (FORMAT JSON) " + query)
    plan = cursor.fetchall()

In [5]:
plan

[([{'Plan': {'Node Type': 'Hash Join',
     'Parallel Aware': False,
     'Async Capable': False,
     'Join Type': 'Inner',
     'Startup Cost': 13.82,
     'Total Cost': 5499.39,
     'Plan Rows': 150000,
     'Plan Width': 593,
     'Inner Unique': True,
     'Hash Cond': '(c.c_nationkey = n.n_nationkey)',
     'Plans': [{'Node Type': 'Seq Scan',
       'Parent Relationship': 'Outer',
       'Parallel Aware': False,
       'Async Capable': False,
       'Relation Name': 'customer',
       'Alias': 'c',
       'Startup Cost': 0.0,
       'Total Cost': 5082.0,
       'Plan Rows': 150000,
       'Plan Width': 159},
      {'Node Type': 'Hash',
       'Parent Relationship': 'Inner',
       'Parallel Aware': False,
       'Async Capable': False,
       'Startup Cost': 11.7,
       'Total Cost': 11.7,
       'Plan Rows': 170,
       'Plan Width': 434,
       'Plans': [{'Node Type': 'Seq Scan',
         'Parent Relationship': 'Outer',
         'Parallel Aware': False,
         'Async Capabl